In [42]:
from keras.datasets import boston_housing
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

In [43]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [44]:
print(train_labels[0])

15.2


In [45]:
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [46]:
# Scaling
# 0번 축은 가로, 1번 축은 세로
# 각 column별의 평균을 구하기.

mean = train_data.mean(axis = 0)
train_data -= mean    #train_data = train_data - mean과 같다.
std = train_data.std(axis = 0)
train_data /= std

train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [53]:
# 모델 구성하기
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)))
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(1))      #집값 하나만 나오기 때문에 1개, 회귀값이기때문에 활성함수를 지정하지 않아도 된다.
  model.compile(optimizer = 'rmsprop',
                loss = 'mse',
                metrics = ['mse'])
  return model

In [59]:
# K-folder 검증을 사용한 훈련 검증

import numpy as np

k = 4                                         # folder 갯수
num_val_samples = len(train_data) // k
all_scores = []

for i in range(k):
  print('처리중인 폴드는 #', i)

  #검증 데이터 준비
  val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples] #시작점과 끝점
  val_labels = train_labels[i * num_val_samples : (i + 1) * num_val_samples]

  #학습 데이터 준비
  data1 = train_data[ : i * num_val_samples]
  data2 = train_data[(i+1) * num_val_samples : ]
  data1_labels = train_labels[ : i * num_val_samples]
  data2_labels = train_labels[(i+1) * num_val_samples : ]

  partial_train_data = np.concatenate([data1, data2], axis = 0) #같은 구조의 데이터를 합쳐주는 함수.
  partial_train_labels = np.concatenate([data1_labels, data2_labels], axis = 0)

  # 모델 학습하기
  model = build_model()
  model.summary() #parameter를 보면 모델의 복잡성을 알 수 있다

  model.fit(partial_train_data, partial_train_labels, epochs = 500, batch_size = 128, verbose = 0) #0이면 훈련사항을 보여주지 x, 기본값은 1

  #모델 검증하기
  val_mse, val_mae = model.evaluate(val_data, val_labels)
  print(val_mse, val_mae)
  all_scores.append(val_mae)

처리중인 폴드는 # 0
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 64)                896       
                                                                 
 dense_30 (Dense)            (None, 64)                4160      
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 3ms/step - loss: 6.3673 - mse: 6.3673
6.367284774780273 6.367284774780273
처리중인 폴드는 # 1
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense) 

In [58]:
np.mean(all_scores)

10.687014818191528